In [1]:
import json
import os
import shutil

In [2]:
svcomp_results = open("svcomp_uautomizer_reachsafety_neat.json", "r")
svcomp_results_json = json.load(svcomp_results)
svcomp_results.close()

In [3]:
svcomp_benchmarks = svcomp_results_json["rows"]

In [56]:
svcomp_benchmarks_dict = []
for benchmark in svcomp_benchmarks:
    if (
        benchmark["id"][1] == "true"
        and benchmark["results"][0]["values"][0]["raw"] != "true"
    ):
        svcomp_benchmarks_dict.append(benchmark["href"])

In [57]:
len(svcomp_benchmarks_dict)

2211

In [8]:
svcomp_benchmarks_dict = ["../" + file[:-4] + ".c" for file in svcomp_benchmarks_dict]

In [11]:
with open("../experiments/svcomp_files.txt", "w") as f:
    for file in svcomp_benchmarks_dict:
        f.write(file + "\n")

In [31]:
svcomp_uautomizer_categories = {}
for benchmark in svcomp_benchmarks_dict:
    svcomp_uautomizer_categories[benchmark["results"][0]["values"][0]["raw"]] = True

In [32]:
svcomp_failure_files = []
for benchmark in svcomp_benchmarks_dict:
    svcomp_failure_files.append(benchmark["href"])

In [33]:
svcomp_failure_files = ["../" + file for file in svcomp_failure_files]

In [34]:
svcomp_failure_files = [file[:-4] + ".c" for file in svcomp_failure_files]

In [35]:
with open("svcomp_reachsafety_files.txt", "w") as f:
    for file in svcomp_failure_files:
        f.write(file + "\n")

In [36]:
t_file = open("../experiments/svcomp_reachsafety_filtered.txt", "r")
files = t_file.read().split("\n")
t_file.close()

In [44]:
new_files = []

In [45]:
for file in files:
    new_file_folder = "../data/benchmarks/" + file[6:]
    new_file_folder = new_file_folder.replace("/c/", "/new-categories/")
    new_file_loc = new_file_folder.split("/")[-1]
    new_file_folder = "/".join(new_file_folder.split("/")[:-1])
    if not os.path.exists(new_file_folder):
        os.makedirs(new_file_folder)
    # shutil.copy(file, new_file_folder + "/" + new_file_loc)
    new_files.append(new_file_folder + "/" + new_file_loc)

In [48]:
with open("../experiments/svcomp_reachsafety_filtered.txt", "w") as f:
    files = "\n".join(new_files)
    f.write(files)

In [2]:
import json

In [17]:
svcomp_features = json.load(open("svcomp_features.json", "r"))

In [ ]:
valid_files = []

for file in svcomp_features:
    if svcomp_features[file]["category"] == "reachability":
        valid_files.append(file)

In [5]:
all_svcomp_files = json.load(open("../../../svcomp_reachability_features.json", "r"))

In [10]:
svcomp_termination_files = json.load(open("svcomp_termination_neat.json", "r"))

term_targets = {}

for file in svcomp_termination_files["rows"]:
    num_success = [
        tool["values"][0]["raw"] == "true" for tool in file["results"]
    ].count(True)
    term_targets["../" + file["href"][:-4] + ".c"] = num_success

In [6]:
targets = {}

for file in all_svcomp_files["rows"]:
    num_success = [
        tool["values"][0]["raw"] == "true" for tool in file["results"]
    ].count(True)
    targets["../" + file["href"][:-4] + ".c"] = num_success

In [20]:
with open("number_of_tools_solving_svcomp.json", "w") as f:
    json.dump(targets, f, indent=4, ensure_ascii=False)

In [11]:
with open("number_of_tools_solving_svcomp_termination.json", "w") as f:
    json.dump(term_targets, f, indent=4, ensure_ascii=False)

In [9]:
tool_names = [x[0] for x in all_svcomp_files["head"]["tool"]["content"]]

In [12]:
num_tools = len(tool_names)

In [69]:
final_json = []
for row in all_svcomp_files["rows"]:
    benchmark_json = {
        "file": row["id"][0],
        "groundtruth": row["id"][1],
        "tool_results": [],
    }
    for tool in row["results"]:
        tool_json = {"tool": tool["href"], "result": tool["values"][0]["raw"]}
        benchmark_json["tool_results"].append(tool_json)
    final_json.append(benchmark_json)

In [70]:
with open("svcomp_all_tools.json", "w") as f:
    json.dump(final_json, f, indent=4)

In [35]:
def any(x):
    for i in x:
        if i:
            return True
    return False

In [73]:
all_tools_failed_benchmarks = []
ultimate_failed_benchmarks = []

for benchmark in final_json:
    if benchmark["groundtruth"] == "true":
        no_tool = all([x["result"] != "true" for x in benchmark["tool_results"]])
        not_ultimate = False
        for tool in benchmark["tool_results"]:
            if tool["tool"].startswith("uautomizer"):
                not_ultimate = tool["result"] != "true"
        if no_tool:
            all_tools_failed_benchmarks.append(benchmark["file"])
        if not_ultimate:
            ultimate_failed_benchmarks.append(benchmark["file"])

In [74]:
len(ultimate_failed_benchmarks), len(all_tools_failed_benchmarks)

(2211, 623)

In [76]:
ultimate_c = [
    "../../sv-benchmarks/c/" + x[:-4] + ".c" for x in ultimate_failed_benchmarks
]
with open("ultimate_failed_benchmarks.txt", "w") as f:
    f.write("\n".join(ultimate_c))

In [48]:
len(all_tools_failed_benchmarks)

623

In [75]:
all_c = ["../../sv-benchmarks/c/" + x[:-4] + ".c" for x in all_tools_failed_benchmarks]
with open("all_tools_failed.txt", "w") as f:
    f.write("\n".join(all_c))

In [21]:
all_json = json.load(open("all_svcomp_features.json", "r"))

In [39]:
positive_targets = []
ultimate = 0

for b in all_json:
    if b["num_loops"] == None:
        continue
    if (
        (b["num_loops"] > 1 or b["more_than_one_method"])
        and (not b["uses_arrays"])
        and (not b["uses_pointers"])
        and b["Ground Truth"]
        and b["num_lines"] <= 500
    ):
        if b["Ultimate Automizer Fails"]:
            ultimate += 1
        positive_targets.append((b["file"], b["Number of Solvers"]))

In [40]:
ultimate

233

In [41]:
ftargets = []

for b in positive_targets:
    if b[1] >= 5:
        ftargets.append(b[0])

In [43]:
len(ftargets)

360

In [44]:
with open("at_least_5_tools.txt", "w") as f:
    f.write("\n".join(ftargets))